In [9]:
import os
import gc

# Set loglevel to suppress tensorflow GPU messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
import datasets
from change_detector import ChangeDetector
from image_translation import ImageTranslationNetwork
from change_priors import Degree_matrix, ztz, image_in_patches
from config import get_config_kACE
from decorators import image_to_tensorboard
import numpy as np

from Code_Aligned_Autoencoders import Kern_AceNet

In [16]:
DATASET="Texas"
CONFIG = get_config_kACE(DATASET)
x_im, y_im, EVALUATE, (C_X, C_Y) = datasets.fetch(DATASET, dim=1, crop_image=False, **CONFIG)


fetch
texas: (1534, 808, 7) (1534, 808, 10)
clipping
Picking dim 2
Loading dataset.. x,y,changemap (1534, 808, 1) (1534, 808, 1) (1534, 808, 1)
dataset: tf.Tensor(-0.566398, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [17]:
#float32
print(x_im.shape, y_im.shape, C_X, C_Y)
print(tf.math.reduce_min(x_im))
print(tf.math.reduce_max(x_im))
x_im.shape[:-1]
Pu = tf.expand_dims(tf.ones(x_im.shape[:-1], dtype=tf.float32), -1) #everyone but the last axix
#creates an input of 1s like (1, 154, 82, 1)
print(Pu.shape)
print(x_im.shape, y_im.shape, C_X, C_Y)
print("EVALUATE", EVALUATE) #master, slave, ground truth pixels (true false)
print(tf.math.reduce_min(x_im))
print(tf.math.reduce_max(x_im))


(1, 1534, 808, 1) (1, 1534, 808, 1) 1 1
tf.Tensor(-0.566398, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)
(1, 1534, 808, 1)
(1, 1534, 808, 1) (1, 1534, 808, 1) 1 1
EVALUATE <TensorSliceDataset shapes: ((1534, 808, 1), (1534, 808, 1), (1534, 808, 1)), types: (tf.float32, tf.float32, tf.bool)>
tf.Tensor(-0.566398, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [18]:
# CPU or GPU selection
if tf.config.list_physical_devices("GPU") and not CONFIG["debug"]:
    C_CODE = 3
    print("here")
    TRANSLATION_SPEC = {
        "enc_X": {"input_chs": C_X, "filter_spec": [50, 50, C_CODE]},
        "enc_Y": {"input_chs": C_Y, "filter_spec": [50, 50, C_CODE]},
        "dec_X": {"input_chs": C_CODE, "filter_spec": [50, 50, C_X]},
        "dec_Y": {"input_chs": C_CODE, "filter_spec": [50, 50, C_Y]},
    }
else:
    print("why here?")
    C_CODE = 1
    TRANSLATION_SPEC = {
        "enc_X": {"input_chs": C_X, "filter_spec": [C_CODE]},
        "enc_Y": {"input_chs": C_Y, "filter_spec": [C_CODE]},
        "dec_X": {"input_chs": C_CODE, "filter_spec": [C_X]},
        "dec_Y": {"input_chs": C_CODE, "filter_spec": [C_Y]},
    }

why here?


In [19]:
# Initialization
print("Change Detector Init")
cd = Kern_AceNet(TRANSLATION_SPEC, **CONFIG)

Change Detector Init


In [20]:
# Training
print("Training")
training_time = 0
cross_loss_weight = tf.expand_dims(tf.zeros(x_im.shape[:-1], dtype=tf.float32), -1)
print(cross_loss_weight.shape)
for epochs in CONFIG["list_epochs"]:
    CONFIG.update(epochs=epochs)
    tr_gen, dtypes, shapes = datasets._training_data_generator(
        x_im[0], y_im[0], cross_loss_weight[0], CONFIG["patch_size"]
    )
    TRAIN = tf.data.Dataset.from_generator(tr_gen, dtypes, shapes)
    TRAIN = TRAIN.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    tr_time, _ = cd.train(TRAIN, evaluation_dataset=EVALUATE, **CONFIG)
    for x, y, _ in EVALUATE.batch(1):
        alpha = cd([x, y])
    cross_loss_weight = 1.0 - alpha
    training_time += tr_time


  0%|          | 0/2 [00:00<?, ?it/s]Training
(1, 1534, 808, 1)
cohens kappa: -0.13181484
total_loss 1.4854898 Target 1.4854898 Left 10
 50%|█████     | 1/2 [00:07<00:07,  7.75s/it]cohens kappa: -0.13210654
total_loss 1.0496955 Target 1.0496955 Left 10
  0%|          | 0/2 [00:00<?, ?it/s]cohens kappa: -0.13232791
total_loss 1.4315213 Target 1.4315213 Left 10
 50%|█████     | 1/2 [00:03<00:03,  3.77s/it]cohens kappa: -0.1324414
total_loss 2.274427 Target 1.4315213 Left 9
  0%|          | 0/1 [00:00<?, ?it/s]cohens kappa: -0.13251781
total_loss 1.3018103 Target 1.3018103 Left 10
100%|██████████| 1/1 [00:03<00:00,  3.85s/it]


In [21]:
cd.load_all_weights(cd.log_path)
cd.final_evaluate(EVALUATE, **CONFIG)
final_kappa = cd.metrics_history["cohens kappa"][-1]
final_acc = cd.metrics_history["ACC"][-1]
performance = (final_kappa, final_acc)
timestamp = cd.timestamp
epoch = cd.epoch.numpy()
speed = (epoch, training_time, timestamp)
del cd
gc.collect()
print(performance, speed)

cohens kappa: -0.07921803
(-0.07921803, 0.81720036) (5, 23.00617996800429, '20210107-131049')
